---

## Ejercicios autónomos (40 minutos)

A continuación, te proponemos ejercicios prácticos para que pongas en práctica lo aprendido sobre la arquitectura Transformer, la librería Transformers de Hugging Face y el uso de pipelines en tareas de PLN.

**Recorda:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y busca modelos que sean apropiados para español, o incluso para el contexto argentino si los encontras.

### 1. Clasificación de textos propios
- Escribi dos textos breves (pueden ser opiniones, quejas o comentarios sobre servicios o productos en Argentina, por ejemplo sobre colectivos, supermercados, bancos, etc.).
- Busca en Hugging Face un modelo de clasificación de texto en español y usalo con el pipeline `text-classification` para analizar el sentimiento de cada texto.
- ¿El resultado coincide con tu expectativa? Explica brevemente.

### 2. Reconocimiento de entidades en noticias argentinas
- Busca un párrafo de una noticia reciente de un medio argentino (por ejemplo, Clarín, La Nación, Página/12).
- Busca en Hugging Face un modelo de NER para español y aplícalo para identificar entidades nombradas.
- Enumera las entidades encontradas y clasifícalas (persona, organización, lugar, etc.).

### 3. Respuesta a preguntas personalizada
- Escribi un pequeño texto narrativo (4-5 líneas) sobre una situación cotidiana en Buenos Aires (por ejemplo, un reclamo en una oficina pública, una experiencia en el subte, etc.).
- Formula dos preguntas sobre el texto y utiliza un modelo de `question-answering` de Hugging Face para responderlas.
- ¿Las respuestas son correctas y precisas? ¿Por qué?

### 4. Resumen de un texto propio
- Elegí un texto más largo (puede ser un fragmento de Wikipedia sobre Argentina, una noticia, etc.).
- Busca un modelo de resumen en español en Hugging Face y úsalo con el pipeline `summarization`.
- Compara el resumen generado con el texto original: ¿qué información se perdió y cuál se mantuvo?

### 5. Traducción y análisis
- Escribi un texto breve en español rioplatense y tradúcilo al inglés usando un modelo de traducción de Hugging Face.
- Luego, traduci el resultado nuevamente al español (podes usar otro pipeline o Google Translate).
- ¿El texto final es igual al original? ¿Qué diferencias encontras?

### 6. Generación creativa de texto
- Escribe el inicio de una historia o una pregunta abierta relacionada con Buenos Aires.
- Busca un modelo de generación de texto en español en Hugging Face y úsalo con el pipeline `text-generation` para continuar el texto.
- Analiza la coherencia y creatividad de la respuesta generada.

---

**Tip:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y lee la descripción de los modelos antes de usarlos. Si tienes dudas, consulta la documentación oficial de [Transformers](https://huggingface.co/docs/transformers/index).

In [19]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Cargar el modelo NER
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

# Cargar el modelo de traducción
translator = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")

# Cargar el modelo de question answering
qa_pipeline = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")

# Cargar el modelo de generación de texto
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")

# Modelo y tokenizer para resumen
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu
Device set to use cpu


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu


# **1. Clasificación de textos propios**


In [2]:
# Inicializamos el pipeline de clasificación
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")

# Textos a analizar
texto1 = "El colectivo que tomé hoy estaba sucio y tardó mucho en llegar. Además, el conductor no paró en la parada indicada."
texto2 = "La atención en el supermercado fue excelente, los cajeros muy rápidos y amables."

# Clasificación de los textos
resultado1 = classifier(texto1)[0]
resultado2 = classifier(texto2)[0]

# Mostramos los resultados de forma prolija
print(f"- Texto 1:\n\"{texto1}\"\n - Sentimiento: {resultado1['label']} (confianza: {round(resultado1['score']*100, 2)}%)\n")
print(f"- Texto 2:\n\"{texto2}\"\n - Sentimiento: {resultado2['label']} (confianza: {round(resultado2['score']*100, 2)}%)")


Device set to use cpu


- Texto 1:
"El colectivo que tomé hoy estaba sucio y tardó mucho en llegar. Además, el conductor no paró en la parada indicada."
 - Sentimiento: NEG (confianza: 92.66%)

- Texto 2:
"La atención en el supermercado fue excelente, los cajeros muy rápidos y amables."
 - Sentimiento: POS (confianza: 97.21%)


# **2. Reconocimiento de entidades en noticias argentinas**

In [6]:
# Texto de una noticia argentina (Ejemplo)
texto_noticia = """El presidente Alberto Fernández anunció un nuevo plan de infraestructura en la ciudad de Buenos Aires, que incluye la construcción de 50 kilómetros de nuevas autopistas."""

# Reconocimiento de entidades
entidades = ner_tagger(texto_noticia)

# Salida formateada
print(" Texto:")
print(f"{texto_noticia}\n")
print(" Entidades reconocidas:")

for entidad in entidades:
    print(f"• {entidad['word']} → {entidad['entity_group']} (confianza: {round(entidad['score']*100, 2)}%)")


 Texto:
El presidente Alberto Fernández anunció un nuevo plan de infraestructura en la ciudad de Buenos Aires, que incluye la construcción de 50 kilómetros de nuevas autopistas.

 Entidades reconocidas:
• Alberto Fernández → PER (confianza: 99.95999908447266%)
• Buenos Aires → LOC (confianza: 99.94000244140625%)


# **3. Respuesta a preguntas personalizada**

In [9]:
# Texto narrativo
texto_narrativo = (
    "Estaba en la oficina pública para renovar mi DNI, pero había una fila enorme. "
    "Después de esperar una hora, finalmente pude entregar los papeles y me dijeron que debía volver otro día."
)

# Preguntas
preguntas = {
    "¿Qué tenía que hacer el narrador?": None,
    "¿Cuánto tiempo esperó el narrador?": None
}

# Responder las preguntas
for pregunta in preguntas:
    respuesta = qa_pipeline(question=pregunta, context=texto_narrativo)
    preguntas[pregunta] = respuesta

# Mostrar resultados de forma clara
print(" Texto narrativo:")
print(texto_narrativo + "\n")

print(" Respuestas obtenidas:")

for pregunta, respuesta in preguntas.items():
    print(f"• {pregunta}\n   Respuesta: {respuesta['answer']} (confianza: {round(respuesta['score']*100, 2)}%)\n")

 Texto narrativo:
Estaba en la oficina pública para renovar mi DNI, pero había una fila enorme. Después de esperar una hora, finalmente pude entregar los papeles y me dijeron que debía volver otro día.

 Respuestas obtenidas:
• ¿Qué tenía que hacer el narrador?
   Respuesta: entregar los papeles (confianza: 34.57%)

• ¿Cuánto tiempo esperó el narrador?
   Respuesta: una hora (confianza: 100.0%)



# **4. Resumen de un texto propio**

In [15]:
# Texto largo
texto_largo = """Argentina es un país de América del Sur que limita con Chile al oeste, Bolivia y Paraguay al norte, Brasil al noreste, Uruguay y el océano Atlántico al este. Tiene una gran diversidad de paisajes y una vasta historia. El país es conocido por su cultura vibrante, que incluye el tango, el fútbol y la gastronomía, como el famoso asado."""

# Generar resumen
resumen = resumidor(texto_largo, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])


El gobierno de Argentina anunció este miércoles la creación de un comité nacional para el Mundial de Fútbol.


# **5. Traducción y análisis**

In [17]:
# Texto en español
texto_espanol = "El colectivo en Buenos Aires siempre está lleno y no tiene aire acondicionado, lo que lo hace muy incómodo."

# Traducir al inglés
traduccion = translator(texto_espanol)
print(traduccion[0]['translation_text'])


The collective in Buenos Aires is always full and has no air conditioning, which makes it very uncomfortable.


# **6. Generación creativa de texto**

In [20]:
# Texto inicial
inicio_historia = "Era un día caluroso de verano en Buenos Aires, y decidí dar un paseo por el barrio de San Telmo..."

# Generar el texto
continuacion = generator(inicio_historia, max_new_tokens=150, do_sample=True, temperature=0.7)
print(continuacion[0]['generated_text'])


Era un día caluroso de verano en Buenos Aires, y decidí dar un paseo por el barrio de San Telmo...

"La plaza de San Telmo o "La plaza del barrio" es un paseo de ronda por la calle Buenos Aires, el barrio del barrio de Buenos Aires.

El Paseo de la Castellana es un paseo por la calle Madrid, el barrio de San Telmo o "La plaza del barrio" es un paseo por la calle Madrid, el barrio de San Telmo o "La plaza del barrio" es un paseo por la calle Madrid, el barrio de San Telmo o "La plaza del barrio" es un paseo por la calle Madrid, el barrio de San Telmo o "La plaza del barrio" es un paseo por la calle Madrid, el barrio de San Telmo o "La plaza del barrio" es un paseo por la calle Madrid, el
